In [1]:
import sys
sys.path.append('../')
%reload_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin

import utils.paramUtil as paramUtil
from options.train_options import TrainCompOptions
from utils.plot_script import *

from models import MotionTransformer
from trainers import DDPMTrainer
from datasets import Text2MotionDataset

from mmcv.runner import get_dist_info, init_dist
from mmcv.parallel import MMDistributedDataParallel, MMDataParallel
import torch
import torch.distributed as dist

In [2]:
class Temp:
    def __init__(self):
        print('mock:: opt')
opt = Temp()

def build_models(opt, dim_pose):
    encoder = MotionTransformer(
        input_feats=dim_pose,
        num_frames=opt.max_motion_length,
        num_layers=opt.num_layers,
        latent_dim=opt.latent_dim,
        no_clip=opt.no_clip,
        no_eff=opt.no_eff)
    return encoder

# mock from /home/epinyoan/git/MotionDiffuse/text2motion/options/train_options.py
opt.num_layers = 8
opt.latent_dim = 512
opt.diffusion_steps = 1000
opt.no_clip = False
opt.no_eff = False
opt.num_epochs = 50
opt.lr = 2e-4
opt.batch_size = 32
opt.times = 1
opt.feat_bias = 1
opt.is_continue = False
opt.log_every = 50
opt.save_every_e = 5
opt.log_every = 5
opt.eval_every_e = 5
opt.save_latest = 500
opt.is_train = True

# mock from cmd
opt.name = 'kit_baseline_dp_2gpu_8layers_1000'
opt.batch_size = 128
opt.times = 50
opt.num_epochs = 50
opt.dataset_name = 'kit'
opt.num_layers = 8
opt.diffusion_steps = 1000
opt.data_parallel = True
opt.gpu_id = [0,1,2,3]
opt.device = torch.device("cuda")

# mock from /home/epinyoan/git/MotionDiffuse/text2motion/options/base_options.py
opt.checkpoints_dir = '../checkpoints'
opt.distributed = False

opt.save_root = pjoin(opt.checkpoints_dir, opt.dataset_name, opt.name)
opt.model_dir = pjoin(opt.save_root, 'model')
opt.meta_dir = pjoin(opt.save_root, 'meta')
opt.data_root = '../data/HumanML3D' # change root to '../'
opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
opt.text_dir = pjoin(opt.data_root, 'texts')
opt.joints_num = 22
radius = 4
fps = 20
opt.max_motion_length = 196
dim_pose = 263
kinematic_chain = paramUtil.t2m_kinematic_chain

dim_word = 300
mean = np.load(pjoin(opt.data_root, 'Mean.npy'))
std = np.load(pjoin(opt.data_root, 'Std.npy'))

train_split_file = pjoin(opt.data_root, 'train.txt')
encoder = build_models(opt, dim_pose)
encoder = MMDataParallel(
            encoder.cuda(opt.gpu_id[0]), device_ids=opt.gpu_id)

train_dataset = Text2MotionDataset(opt, mean, std, train_split_file, opt.times)

mock:: opt


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23384/23384 [00:08<00:00, 2746.05it/s]


In [3]:
train_dataset[329]

('a person quickly changing direction while running from right to left, as if they wanted to dodge something',
 array([[-2.97006446e+00, -2.04159292e+00,  5.68988223e+00, ...,
          3.91285052e-01, -2.38431600e+00, -2.41930076e+00],
        [-6.55511139e-02, -2.09677192e+00,  5.82259087e+00, ...,
          3.91285052e-01, -2.38431600e+00, -2.41930076e+00],
        [-7.16576839e-02, -1.93841658e+00,  6.49210426e+00, ...,
         -2.41837683e+00, -2.38431600e+00, -2.41930076e+00],
        ...,
        [ 4.40691392e-04,  1.42151756e-03, -5.12978332e-01, ...,
         -2.41837683e+00, -2.38431600e+00, -2.41930076e+00],
        [ 4.40691392e-04,  1.42151756e-03, -5.12978332e-01, ...,
         -2.41837683e+00, -2.38431600e+00, -2.41930076e+00],
        [ 4.40691392e-04,  1.42151756e-03, -5.12978332e-01, ...,
         -2.41837683e+00, -2.38431600e+00, -2.41930076e+00]]),
 32)

In [6]:
del trainer
trainer = DDPMTrainer(opt, encoder)
trainer.train(train_dataset)

epoch:   0 niter:      5  inner_iter:    4 0m 17s loss_mot_rec: 0.9901 
epoch:   0 niter:     10  inner_iter:    9 0m 25s loss_mot_rec: 0.9637 
epoch:   0 niter:     15  inner_iter:   14 0m 33s loss_mot_rec: 0.9361 
epoch:   0 niter:     20  inner_iter:   19 0m 42s loss_mot_rec: 0.9096 
epoch:   0 niter:     25  inner_iter:   24 0m 50s loss_mot_rec: 0.8858 
epoch:   0 niter:     30  inner_iter:   29 0m 58s loss_mot_rec: 0.8576 
epoch:   0 niter:     35  inner_iter:   34 1m 7s loss_mot_rec: 0.8278 
epoch:   0 niter:     40  inner_iter:   39 1m 15s loss_mot_rec: 0.8016 
epoch:   0 niter:     45  inner_iter:   44 1m 24s loss_mot_rec: 0.7732 
epoch:   0 niter:     50  inner_iter:   49 1m 32s loss_mot_rec: 0.7466 
epoch:   0 niter:     55  inner_iter:   54 1m 40s loss_mot_rec: 0.7194 
epoch:   0 niter:     60  inner_iter:   59 1m 49s loss_mot_rec: 0.6925 
epoch:   0 niter:     65  inner_iter:   64 1m 57s loss_mot_rec: 0.6673 
epoch:   0 niter:     70  inner_iter:   69 2m 5s loss_mot_rec: 0.

epoch:   0 niter:    575  inner_iter:  574 16m 5s loss_mot_rec: 0.0937 
epoch:   0 niter:    580  inner_iter:  579 16m 13s loss_mot_rec: 0.0982 
epoch:   0 niter:    585  inner_iter:  584 16m 21s loss_mot_rec: 0.1024 
epoch:   0 niter:    590  inner_iter:  589 16m 29s loss_mot_rec: 0.0882 
epoch:   0 niter:    595  inner_iter:  594 16m 38s loss_mot_rec: 0.0913 
epoch:   0 niter:    600  inner_iter:  599 16m 46s loss_mot_rec: 0.0936 
epoch:   0 niter:    605  inner_iter:  604 16m 54s loss_mot_rec: 0.0904 
epoch:   0 niter:    610  inner_iter:  609 17m 3s loss_mot_rec: 0.0857 
epoch:   0 niter:    615  inner_iter:  614 17m 11s loss_mot_rec: 0.0880 
epoch:   0 niter:    620  inner_iter:  619 17m 19s loss_mot_rec: 0.0920 
epoch:   0 niter:    625  inner_iter:  624 17m 27s loss_mot_rec: 0.0855 
epoch:   0 niter:    630  inner_iter:  629 17m 36s loss_mot_rec: 0.0927 
epoch:   0 niter:    635  inner_iter:  634 17m 44s loss_mot_rec: 0.0854 
epoch:   0 niter:    640  inner_iter:  639 17m 52s lo

epoch:   0 niter:   1140  inner_iter: 1139 31m 44s loss_mot_rec: 0.0621 
epoch:   0 niter:   1145  inner_iter: 1144 31m 53s loss_mot_rec: 0.0575 
epoch:   0 niter:   1150  inner_iter: 1149 32m 1s loss_mot_rec: 0.0590 
epoch:   0 niter:   1155  inner_iter: 1154 32m 9s loss_mot_rec: 0.0596 
epoch:   0 niter:   1160  inner_iter: 1159 32m 17s loss_mot_rec: 0.0539 
epoch:   0 niter:   1165  inner_iter: 1164 32m 26s loss_mot_rec: 0.0527 
epoch:   0 niter:   1170  inner_iter: 1169 32m 34s loss_mot_rec: 0.0525 
epoch:   0 niter:   1175  inner_iter: 1174 32m 42s loss_mot_rec: 0.0539 
epoch:   0 niter:   1180  inner_iter: 1179 32m 50s loss_mot_rec: 0.0515 
epoch:   0 niter:   1185  inner_iter: 1184 32m 58s loss_mot_rec: 0.0609 
epoch:   0 niter:   1190  inner_iter: 1189 33m 7s loss_mot_rec: 0.0550 
epoch:   0 niter:   1195  inner_iter: 1194 33m 15s loss_mot_rec: 0.0544 
epoch:   0 niter:   1200  inner_iter: 1199 33m 23s loss_mot_rec: 0.0547 
epoch:   0 niter:   1205  inner_iter: 1204 33m 32s los

epoch:   0 niter:   1705  inner_iter: 1704 47m 26s loss_mot_rec: 0.0458 
epoch:   0 niter:   1710  inner_iter: 1709 47m 35s loss_mot_rec: 0.0483 
epoch:   0 niter:   1715  inner_iter: 1714 47m 44s loss_mot_rec: 0.0470 
epoch:   0 niter:   1720  inner_iter: 1719 47m 52s loss_mot_rec: 0.0451 
epoch:   0 niter:   1725  inner_iter: 1724 48m 1s loss_mot_rec: 0.0484 
epoch:   0 niter:   1730  inner_iter: 1729 48m 9s loss_mot_rec: 0.0455 
epoch:   0 niter:   1735  inner_iter: 1734 48m 18s loss_mot_rec: 0.0385 
epoch:   0 niter:   1740  inner_iter: 1739 48m 26s loss_mot_rec: 0.0479 
epoch:   0 niter:   1745  inner_iter: 1744 48m 34s loss_mot_rec: 0.0466 
epoch:   0 niter:   1750  inner_iter: 1749 48m 43s loss_mot_rec: 0.0444 
epoch:   0 niter:   1755  inner_iter: 1754 48m 51s loss_mot_rec: 0.0455 
epoch:   0 niter:   1760  inner_iter: 1759 48m 59s loss_mot_rec: 0.0492 
epoch:   0 niter:   1765  inner_iter: 1764 49m 7s loss_mot_rec: 0.0516 
epoch:   0 niter:   1770  inner_iter: 1769 49m 15s los

epoch:   0 niter:   2270  inner_iter: 2269 63m 7s loss_mot_rec: 0.0429 
epoch:   0 niter:   2275  inner_iter: 2274 63m 16s loss_mot_rec: 0.0407 
epoch:   0 niter:   2280  inner_iter: 2279 63m 24s loss_mot_rec: 0.0434 
epoch:   0 niter:   2285  inner_iter: 2284 63m 32s loss_mot_rec: 0.0415 
epoch:   0 niter:   2290  inner_iter: 2289 63m 41s loss_mot_rec: 0.0427 
epoch:   0 niter:   2295  inner_iter: 2294 63m 49s loss_mot_rec: 0.0435 
epoch:   0 niter:   2300  inner_iter: 2299 63m 57s loss_mot_rec: 0.0432 
epoch:   0 niter:   2305  inner_iter: 2304 64m 5s loss_mot_rec: 0.0420 
epoch:   0 niter:   2310  inner_iter: 2309 64m 14s loss_mot_rec: 0.0396 
epoch:   0 niter:   2315  inner_iter: 2314 64m 22s loss_mot_rec: 0.0406 
epoch:   0 niter:   2320  inner_iter: 2319 64m 30s loss_mot_rec: 0.0383 
epoch:   0 niter:   2325  inner_iter: 2324 64m 39s loss_mot_rec: 0.0409 
epoch:   0 niter:   2330  inner_iter: 2329 64m 47s loss_mot_rec: 0.0432 
epoch:   0 niter:   2335  inner_iter: 2334 64m 55s lo

KeyboardInterrupt: 